In [2]:
import pandas
import string
import nltk
import keras
import sklearn
import pickle
import numpy as np
import keras.backend as K
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense , Activation , Dropout
from keras.metrics import Precision , Recall , Accuracy , TruePositives , TrueNegatives , FalsePositives , FalseNegatives

# Load Dataset and Mounting Google Drive

ar_reviews_100k.csv

In [3]:
dataset = pandas.read_csv("ar_reviews_100k.tsv", sep='\t', header=0)

In [4]:
print(dataset.head())
print(type(dataset['text']))

      label                                               text
0  Positive  ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1  Positive  أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2  Positive  هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3  Positive  خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4  Positive  ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
<class 'pandas.core.series.Series'>


In [5]:
labels = dataset['label']
reviews = dataset['text']

train_data = dataset.sample(frac=.7)
test_data = dataset.drop(labels=train_data.index)
print(f"Train Data:{len(train_data)}")
print(f"Test Data:{len(test_data)}")

train_labels = train_data['label']
train_reviews = train_data['text']

test_labels =  test_data['label']
test_reviews = test_data['text']

Train Data:69999
Test Data:30000


In [6]:
tokenizer = Tokenizer(num_words=None,lower=False)
tokenizer.fit_on_texts(reviews)

x_train = tokenizer.texts_to_matrix(train_reviews, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_reviews, mode='tfidf')


MemoryError: Unable to allocate 178. GiB for an array with shape (69999, 341256) and data type float64

In [ ]:
encoder = LabelEncoder()
encoder.fit(labels)
labelst=encoder.fit_transform(labels)

num_classes = int((len(set(labelst))))
print((len(set(labelst))))

y_train = encoder.fit_transform(train_labels)
y_test = encoder.fit_transform(test_labels)



In [ ]:
y_train= keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


num_labels = int(len(y_train.shape))
vocab_size = len(tokenizer.word_index) + 1

max_words=vocab_size


In [ ]:

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val




In [ ]:

# Build the model
model = Sequential()
model.add(Dense(1024, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','Recall','Precision', f1_metric,'TruePositives','TrueNegatives','FalsePositives','FalseNegatives'])


In [ ]:
batch_size = 100
epochs = 2

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


model.save('my_model.h1')


In [ ]:


# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
#model = keras.models.load_model('my_model.h1')
Evaluation_valus = model.evaluate(x_test,y_test,verbose=0)
print("Loss" , 'categorical_accuracy','Recall','Precision','f1_metric','TruePositives','TrueNegatives','FalsePositives','FalseNegatives')

print(Evaluation_valus)


In [ ]:
for x in dataset["text"][:25]:

    tokens = tokenizer.reviews_to_matrix([x], mode='tfidf')

    c=model.predict(np.array(tokens))
    cc=model.predict_classes(tokens)
    xc = encoder.inverse_transform(cc)


    print(c,"= \t",cc,"\t",xc)